In [ ]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [ ]:
demographics_event= pd.read_csv('/data/public/MLA/VCHAMPS-Quality/demographics_event_qual.csv').iloc[:,1:]

In [ ]:
demographics_event['Event date'] = demographics_event['Event date'].apply(preprocess_data.preprocess_time_string)

In [ ]:
marital_status = {'Married': 0, 'Divorced': 1, 'Never married': 1, 'Separated': 1, 'Single': 1, 'Unknown': float('nan'), 'Widowed': 1, 'Not specified (no value)': float('nan')}
ruca_category = {'Urban': 0,  'Rural': 1, 'Highly rural': 2, 'Not specified': float('nan')}

In [ ]:
demographics_event['Marital status encoded'] = preprocess_data.preprocess_substitute_categories(marital_status, 
                                                                               demographics_event['Marital status'])
demographics_event['Ruca category encoded'] = preprocess_data.preprocess_substitute_categories(ruca_category, 
                                                                               demographics_event['Ruca category'])